In [1]:
import sys
from pathlib import Path

if 'google.colab' in sys.modules:

    from google.colab import drive
    drive.mount('/content/drive/')
    %cd drive/MyDrive/Kaggle_Feedback-Prize-Evaluating-Student-Writing/main/

    # install packages
    !pip install sentencepiece transformers torch==1.9.1 torchvision==0.10.1 torchAudio==0.9.1 torchtext==0.10.1

Mounted at /content/drive/
/content/drive/MyDrive/Kaggle_Feedback-Prize-Evaluating-Student-Writing/main
     |████████████████████████████████| 1.2 MB 12.1 MB/s 
     |████████████████████████████████| 3.5 MB 54.9 MB/s 
     |████████████████████████████████| 831.4 MB 5.7 kB/s 
     |████████████████████████████████| 22.1 MB 362 kB/s 
     |████████████████████████████████| 1.9 MB 57.2 MB/s 
     |████████████████████████████████| 7.6 MB 30.2 MB/s 
     |████████████████████████████████| 6.8 MB 55.7 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 61.9 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111

# Libraries and Config

In [2]:
import os
import gc
import pandas as pd
import numpy as np
from tqdm import tqdm

from torch import cuda

from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import accuracy_score

In [23]:
class Config:
    nb_name = 'nb006'
    ver = 26
    load_tokens = True # 基本Trueでok
    load_models = True # モデルをロードするときはTrue、学習するときはFalse

    if load_tokens:
        load_tokens_from = f'../input/py-bigbird-v{ver}'
    else:
        load_tokens_from = None

    if load_models:
        load_model_from = f'../input/py-bigbird-v{ver}'
    else:
        load_model_from = None

    downloaded_model_path = f'../input/py-bigbird-v{ver}'

    # testデータが5個以上(提出時)はスコアを計算しない
    compute_val_score = True
    if len(os.listdir('../input/feedback-prize-2021/test'))>5:
        compute_val_score = False


    load_train_text = False # 初回だけTrue

    # os.environ['CUDA_VISIBLE_DEVICES'] = '2'
    device = 'cuda' if cuda.is_available() else 'cpu'

    # fold
    n_fold = 5
    
    # model Config
    model_name = 'google/bigbird-roberta-base'
    max_length = 1024
    train_batch_size = 4
    valid_batch_size = 4
    epochs = 5
    learning_rates = [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7]
    max_grad_norm = 10

print('devide:', Config.device)

devide: cuda


In [5]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(71)

In [6]:
from transformers import *
if Config.downloaded_model_path == 'model':
    os.makedirs(f'{Config.nb_name}_model', exist_ok=True)
    
    tokenizer = AutoTokenizer.from_pretrained(Config.model_name, add_prefix_space=True)
    tokenizer.save_pretrained(f'{Config.nb_name}_model')

    config_model = AutoConfig.from_pretrained(Config.model_name) 
    config_model.num_labels = 15
    config_model.save_pretrained(f'{Config.nb_name}_model')

    backbone = AutoModelForTokenClassification.from_pretrained(Config.model_name, 
                                                               config=config_model)
    backbone.save_pretrained(f'{Config.nb_name}_model')

https://huggingface.co/google/bigbird-roberta-base/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmprnucm759


Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

storing https://huggingface.co/google/bigbird-roberta-base/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/d20a688e918d227ce5dbcd5f2b570a093cee6b095952d74b9c245b245e6510de.c8f14f85d9ff88cdd1fe7094cde11f85b74fcb7eb03616822964895bc6626c3b
creating metadata file for /root/.cache/huggingface/transformers/d20a688e918d227ce5dbcd5f2b570a093cee6b095952d74b9c245b245e6510de.c8f14f85d9ff88cdd1fe7094cde11f85b74fcb7eb03616822964895bc6626c3b
https://huggingface.co/google/bigbird-roberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6wbboj4a


Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

storing https://huggingface.co/google/bigbird-roberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/d7643b757353be56f05bdd19496d6e3fb5bb9edfdf5f9e5eca88d6f479e32324.dc98375bb3e19a644a5cadd5c305949ec470186fcc20bd8c8b959a43dcc3ff21
creating metadata file for /root/.cache/huggingface/transformers/d7643b757353be56f05bdd19496d6e3fb5bb9edfdf5f9e5eca88d6f479e32324.dc98375bb3e19a644a5cadd5c305949ec470186fcc20bd8c8b959a43dcc3ff21
loading configuration file https://huggingface.co/google/bigbird-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/d7643b757353be56f05bdd19496d6e3fb5bb9edfdf5f9e5eca88d6f479e32324.dc98375bb3e19a644a5cadd5c305949ec470186fcc20bd8c8b959a43dcc3ff21
Model config BigBirdConfig {
  "_name_or_path": "google/bigbird-roberta-base",
  "architectures": [
    "BigBirdForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "clas

...

storing https://huggingface.co/google/bigbird-roberta-base/resolve/main/spiece.model in cache at /root/.cache/huggingface/transformers/d318d7bb69cafb1d8964fc87515592ac3092a2c8fdb305068f9ba4020df3ee3b.271d467a9adc15fb44348481bc75c48b63cba0fd4934bc5377d63a63de052c45
creating metadata file for /root/.cache/huggingface/transformers/d318d7bb69cafb1d8964fc87515592ac3092a2c8fdb305068f9ba4020df3ee3b.271d467a9adc15fb44348481bc75c48b63cba0fd4934bc5377d63a63de052c45
https://huggingface.co/google/bigbird-roberta-base/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfadi10o9


Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

storing https://huggingface.co/google/bigbird-roberta-base/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/400be7e354ea6eb77319bcc7fa34899ec9fa2e3aff0fa677f6eb7e45a01b1548.75b358ecb30fa6b001d9d87bfde336c02d9123e7a8f5b90cc890d0f6efc3d4a3
creating metadata file for /root/.cache/huggingface/transformers/400be7e354ea6eb77319bcc7fa34899ec9fa2e3aff0fa677f6eb7e45a01b1548.75b358ecb30fa6b001d9d87bfde336c02d9123e7a8f5b90cc890d0f6efc3d4a3
loading file https://huggingface.co/google/bigbird-roberta-base/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/d318d7bb69cafb1d8964fc87515592ac3092a2c8fdb305068f9ba4020df3ee3b.271d467a9adc15fb44348481bc75c48b63cba0fd4934bc5377d63a63de052c45
loading file https://huggingface.co/google/bigbird-roberta-base/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/google/bigbird-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface

Downloading:   0%|          | 0.00/489M [00:00<?, ?B/s]

storing https://huggingface.co/google/bigbird-roberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/c523b12608662dbff39b2c24a608a6ff30857bc7967a5c9b00cb76d1147e223b.06e7996caf35449212f17d31a2129bb55c59c19054fcf8552a847e4bcb475688
creating metadata file for /root/.cache/huggingface/transformers/c523b12608662dbff39b2c24a608a6ff30857bc7967a5c9b00cb76d1147e223b.06e7996caf35449212f17d31a2129bb55c59c19054fcf8552a847e4bcb475688
loading weights file https://huggingface.co/google/bigbird-roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/c523b12608662dbff39b2c24a608a6ff30857bc7967a5c9b00cb76d1147e223b.06e7996caf35449212f17d31a2129bb55c59c19054fcf8552a847e4bcb475688
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', '

# Load Data

In [7]:
train_df = pd.read_csv('../input/feedback-prize-2021/train.csv')
print(train_df.shape)
train_df.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [8]:
test_names, test_texts = [], []
for f in list(os.listdir('../input/feedback-prize-2021/test')):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id':test_names, 'text':test_texts})
test_texts.head()

,id,text
0,D46BCB48440A,"When people ask for advice,they sometimes talk..."
1,D72CB1C11673,Making choices in life can be very difficult. ...
2,0FB0700DAF44,"During a group project, have you ever asked a ..."
3,DF920E0A7337,Have you ever asked more than one person for h...
4,18409261F5C2,80% of Americans believe seeking multiple opin...


In [9]:
if Config.load_train_text:
    test_names, train_texts = [], []
    for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
        test_names.append(f.replace('.txt', ''))
        train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
    train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
    # train_text_df.to_csv('../input/data/train_text_df.csv', index=False)
else:
    train_text_df = pd.read_csv('../input/fb-data/train_text_df.csv')
train_text_df.head()

,id,text
0,F48EF80D2ED3,There are many programs in the world around yo...
1,F8FB4470A52F,"Dear Senator,\n\n""The Electoral College is a p..."
2,F176A8CF72BB,In my opinion i don't think that is fair. i th...
3,EBDE7FC748A4,Unmasking the Face\n\nThe face on Mars was rea...
4,F6C40C564E5E,Luke think you should join the seagoing cowboy...


# Convert Train Text to NER Labels

In [10]:
if not Config.load_tokens_from:
    all_entities = []
    for i in tqdm(train_text_df.iterrows()):
        total = i[1]['text'].split().__len__()
        entities = ["O"]*total
        for j in train_df[train_df['id'] == i[1]['id']].iterrows():
            discourse = j[1]['discourse_type']
            list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
            entities[list_ix[0]] = f"B-{discourse}"
            for k in list_ix[1:]: entities[k] = f"I-{discourse}"
        all_entities.append(entities)
    train_text_df['entities'] = all_entities
    train_text_df.to_csv('train_NER.csv',index=False)
    
else:
    from ast import literal_eval
    train_text_df = pd.read_csv(f'{Config.load_tokens_from}/train_NER.csv')
    # pandas saves lists as string, we must convert back
    train_text_df.entities = train_text_df.entities.apply(lambda x: literal_eval(x) )
    
print( train_text_df.shape )

(15594, 3)


In [11]:
# create dict
# B: Biginning, I: Inner
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k, v in enumerate(output_labels)}
ids_to_labels = {k:v for k, v in enumerate(output_labels)}

# Dataset

In [12]:
LABEL_ALL_SUBTOKENS = True

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

    def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        text = self.data.text[index]        
        word_labels = self.data.entities[index] if not self.get_wids else None

        # TOKENIZE TEXT
        encoding = self.tokenizer(text.split(),
                             is_split_into_words=True,
                             #return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        word_ids = encoding.word_ids()  
        
        # CREATE TARGETS
        if not self.get_wids:
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:                            
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:              
                    label_ids.append( labels_to_ids[word_labels[word_idx]] )
                else:
                    if LABEL_ALL_SUBTOKENS:
                        label_ids.append( labels_to_ids[word_labels[word_idx]] )
                    else:
                        label_ids.append(-100)
                previous_word_idx = word_idx
            encoding['labels'] = label_ids

        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            word_ids2 = [w if w is not None else -1 for w in word_ids]
            item['wids'] = torch.as_tensor(word_ids2)
        
        return item

    def __len__(self):
        return self.len

# Fold

In [13]:
!pip install -q iterative-stratification
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [14]:
# idごとにdiscourse_typeをグループ化する
train_dfx = pd.get_dummies(train_df, columns=['discourse_type']).groupby(['id'], as_index=False).sum()
# discourse_type_hogeのカラムを取得
cols = [c for c in train_dfx.columns if c.startswith('discourse_type_') or c=='id' and c!='discourse_type_num']
train_dfx = train_dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=71)
# 'discourse_type_Claim'などを取得
labels = [c for c in train_dfx.columns if c!='id']
train_dfx_labels = train_dfx[labels]
train_dfx['kfold'] = -1

for fold, (trn_, val_) in enumerate(mskf.split(train_dfx, train_dfx_labels)):
    print(len(trn_), len(val_))
    train_dfx.loc[val_, 'kfold'] = fold

train_df = train_df.merge(train_dfx[['id', 'kfold']], on='id', how='left')
print(train_df['kfold'].value_counts())
# train_df.to_csv('../input/train_folds.csv', index=False)

12478 3116
12474 3120
12477 3117
12470 3124
12477 3117
2    29082
3    28917
4    28801
1    28786
0    28707
Name: kfold, dtype: int64


# DataLoaders

In [21]:
# train_idx, test_idz
IDS = train_df['id'].unique()
print('There are',len(IDS),'train texts. We will split 90% 10% for validation.')

train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)

There are 15594 train texts. We will split 90% 10% for validation.


In [24]:
# create train subset and valid subset
data = train_text_df[['id','text', 'entities']]
train_dataset = data.loc[data['id'].isin(IDS[train_idx]),['text', 'entities']].reset_index(drop=True)
test_dataset = data.loc[data['id'].isin(IDS[valid_idx])].reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

tokenizer = AutoTokenizer.from_pretrained(Config.downloaded_model_path)
training_set = dataset(train_dataset, tokenizer, Config.max_length, False)
testing_set = dataset(test_dataset, tokenizer, Config.max_length, True)

FULL Dataset: (15594, 3)
TRAIN Dataset: (14034, 2)
TEST Dataset: (1560, 3)


Didn't find file ../input/py-bigbird-v26/added_tokens.json. We won't load it.
loading file ../input/py-bigbird-v26/spiece.model
loading file ../input/py-bigbird-v26/tokenizer.json
loading file None
loading file ../input/py-bigbird-v26/special_tokens_map.json
loading file ../input/py-bigbird-v26/tokenizer_config.json


In [25]:
train_dataset

,text,entities
0,"Dear Principal, I believe in policy 2. Kids ar...","[O, O, B-Position, I-Position, I-Position, I-P..."
1,"Summer projects are no fun, but they are a gre...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
2,"The author who wrote ""The challenge of Explori...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
3,Our school systems have seen many changes as t...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
4,Having a driverless car can have positives and...,"[B-Position, I-Position, I-Position, I-Positio..."
...,...,...
14029,"Nowadays, school can be a very distracting env...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
14030,In my opinion I think that the author did a gr...,"[B-Position, I-Position, I-Position, I-Positio..."
14031,I believe that students should not be allowed ...,"[B-Position, I-Position, I-Position, I-Positio..."
14032,i think that technology is valuable because yo...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."


In [ ]:
train_params = {'batch_size': Config.train_batch_size,
                'shuffle': True,
                'num_workers': 2,
                'pin_memory': True}

test_params = {'batch_size': Config.valid_batch_size,
               'shuffle': False,
               'num_workers': 2,
               'pin_memory': True}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# test dataset
test_texts_set = dataset(test_texts, tokenizer, Config.max_length, True)
test_texts_loader = DataLoader(test_texts_set, **test_params)

# Inference

In [ ]:
def inference(batch):

    # move batch to gpu and infer
    ids = batch['input_ids'].to(Config.device)
    mask = batch['attention_mask'].to(Config.device)
    outputs = model(ids, attention_mask=mask, return_dict=False) # それぞれのidについて推論
    all_preds = torch.argmax(outputs[0], axis=-1).cpu().numpy() # 推論結果をまとめる

    predictions = []
    for k, text_preds in enumerate(all_preds):
        token_preds = [ids_to_labels[i] for i in text_preds] # textからtokenを取得？

        prediction = []
        word_ids = batch['wids'][k].numpy() # validation
        previous_word_idx = -1
        for idx, word_idx in enumerate(word_ids):
            if word_idx == -1:
                pass
            elif word_idx != previous_word_idx:
                prediction.append(token_preds[idx])
                previous_word_idx = word_idx
        predictions.append(prediction)

    return predictions
    

def inference(model, dl, criterion,  valid_flg):
    final_predictiokns = []
    final_predictions_prob = []
    stream = tqdm(dl)
    model.eval()

    valid_loss = 0
    valid_accuracy = 0
    all_logits = None
    for batch_idx, batch in enumerate(stream, start=1):
        ids = batch['input_ids'].to(Config.device, dtype=torch.long)
        mask =


# Fold

In [ ]:
def get_preds_folds(df, dl, criterion, valid_flg=False):
    # create model
    config_model = AutoConfig.from_pretrained(Config.downloaded_model_path+'/config.json')
    model = AutoModelForTokenClassification.from_pretrained(
        Config.downloaded_model_path+'/pytorch_model.bin', config=config_model
    )
    model.to(Config.device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=Config.learning_rates[0])
    model.train()
    for i_fold in range(Config.n_fold):
        model_filename = os.path.join(Config.model_dir, f'{Config.model_savename}_{i_fold}.bin')
        print(f'{model_filename} inference')
        model = model.to(Config.device)
        model

# Train Model

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    #tr_preds, tr_labels = [], []
    
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(Config.device, dtype = torch.long)
        mask = batch['attention_mask'].to(Config.device, dtype = torch.long)
        labels = batch['labels'].to(Config.device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 200==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss after {idx:04d} training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        #tr_labels.extend(labels)
        #tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=Config.max_grad_norm
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")


In [ ]:
# create model
config_model = AutoConfig.from_pretrained(Config.downloaded_model_path+'/config.json')
model = AutoModelForTokenClassification.from_pretrained(
    Config.downloaded_model_path+'/pytorch_model.bin', config=config_model
)
model.to(Config.device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=Config.learning_rates[0])

loading configuration file model/config.json
Model config BigBirdConfig {
  "_name_or_path": "model/config.json",
  "architectures": [
    "BigBirdForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
  

## Train Loop

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
if not Config.load_model_from:
    for epoch in range(Config.epochs):
        
        print(f"### Training epoch: {epoch + 1}")
        for g in optimizer.param_groups: 
            g['lr'] = Config.learning_rates[epoch]
        lr = optimizer.param_groups[0]['lr']
        print(f'### LR = {lr}\n')
        
        train(epoch)
        torch.cuda.empty_cache()
        gc.collect()
        
    torch.save(model.state_dict(), f'bigbird_v{Config.ver}.pt')
else:
    model.load_state_dict(torch.load(f'{Config.load_model_from}/bigbird_v{Config.ver}.pt'))
    print('Model loaded.')

Model loaded.


In [ ]:
def get_predictions(df=test_dataset, loader=testing_loader):
    
    # put model in training mode
    model.eval()
    
    # GET WORD LABEL PREDICTIONS
    y_pred2 = []
    for batch in loader:
        labels = inference(batch)
        y_pred2.extend(labels)

    final_preds2 = []
    for i in range(len(df)):

        idx = df.id.values[i]
        #pred = [x.replace('B-','').replace('I-','') for x in y_pred2[i]]
        pred = y_pred2[i] # Leave "B" and "I"
        preds = []
        j = 0
        while j < len(pred):
            cls = pred[j]
            if cls == 'O': j += 1
            else: cls = cls.replace('B','I') # spans start with B
            end = j + 1
            while end < len(pred) and pred[end] == cls:
                end += 1
            
            if cls != 'O' and cls != '' and end - j > 7:
                final_preds2.append((idx, cls.replace('I-',''),
                                     ' '.join(map(str, list(range(j, end))))))
        
            j = end
        
    oof = pd.DataFrame(final_preds2)
    oof.columns = ['id','class','predictionstring']

    return oof

In [ ]:
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]

def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [ ]:
if Config.compute_val_score: # note this doesn't run during submit
    # VALID TARGETS
    valid = train_df.loc[train_df['id'].isin(IDS[valid_idx])]

    # OOF PREDICTIONS
    oof = get_predictions(test_dataset, testing_loader)

    # COMPUTE F1 SCORE
    f1s = []
    CLASSES = oof['class'].unique()
    print()
    for c in CLASSES:
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = valid.loc[valid['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(c,f1)
        f1s.append(f1)
    print()
    print('Overall',np.mean(f1s))
    print()


Lead 0.893923789907312
Claim 0.6138463210520596
Counterclaim 0.6828828828828829
Rebuttal 0.5990566037735849
Evidence 0.7387687188019967
Concluding Statement 0.872249822569198
Position 0.7605351170568562

Overall 0.7373233222919843



In [ ]:
sub = get_predictions(test_texts, test_texts_loader)
sub.head()

,id,class,predictionstring
0,D46BCB48440A,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1,D46BCB48440A,Position,20 21 22 23 24 25 26 27
2,D46BCB48440A,Evidence,56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 7...
3,D46BCB48440A,Evidence,150 151 152 153 154 155 156 157 158 159 160 16...
4,D46BCB48440A,Evidence,224 225 226 227 228 229 230 231 232 233 234 23...


In [ ]:
# sub.to_csv("submission.csv", index=False)